## Modules

In [1]:

import sys; sys.path.append("../modules")
import random
from itertools import product

import numpy as np
import sympy; sympy.init_printing()
import pandas as pd

import Permutation as pm
#from Grid import *

from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
#help(pd_to_hfk)

from GridPythonModule import *

In [ ]:
count = 0
for _ in tqdm(range(1000)):
    G = generate_random_grid(20)
    if number_of_components(G) == 1:
        # draw_grid(G)
        count += 1
print(count)

## Classes

In [2]:
class Edge:
    '''Edge class expressing edges in a grid diagram.'''
    def __init__(self, hv, index, xoends):
        self.hv = hv
        self.index = index
        self.xoends = xoends
        self.xend, self.oend = xoends
        self.direction = 1 if not (self.xend < self.oend)^(self.hv == 'h') else -1

    def __repr__(self):
        return f"{self.hv}{self.index}" #{self.xoends}"
        
    def __str__(self):
        return f"{self.hv}{self.index}({self.xend},{self.oend})"

class Crossing:
    def __init__(self, hedge, vedge):
        self.hedge, self.vedge = hedge, vedge
        self.sign = self.get_sign()
        self.index = (hedge.index, vedge.index)
        
    def get_sign(self):
        signs = [-1]
        he, ve = self.hedge, self.vedge
        for e, i in [(he, ve.index), (ve, he.index)]:
            e.sorted_end = e.xoends if e.xend < e.oend else e.xoends[::-1]
            sign = e.direction if e.sorted_end[0] < i < e.sorted_end[1] else 0
            signs.append(sign)
        return np.prod(signs)

    def __eq__(self, other_cr):
        return other_cr == self.index

    def __repr__(self):
        return f"x({self.hedge.index},{self.vedge.index})"

class Grid:
    def __init__(self, xlist, olist):
        self.orig = [xlist, olist]
        self.size = len(xlist)
        self.xprm, self.oprm = pm.Permutation(xlist), pm.Permutation(olist)
        self.hedges = [Edge('h', i, (self.xprm.act(i), self.oprm.act(i))) for i in range(self.size)]
        self.vedges = [Edge('v', i, (self.xprm.inverse().act(i), self.oprm.inverse().act(i))) for i in range(self.size)]
        self.crossings = self.get_crossings()
        
    def get_crossings(self):
        crossings = []
        for he, ve in product(self.hedges, self.vedges):
            if not (he.index in [0, self.size-1] or ve.index in [0, self.size-1]):
                cr = Crossing(he, ve)
                crossings.append(cr) if not cr.sign == 0 else None
        return crossings

    def draw(self):
        draw_grid(self.orig, markings='XO')

    def num_of_comps(self):
        number_of_components(self.orig)

    def edge_trace(self):
        et = [self.hedges[0]]
        closed = False
        while not closed:
            et.append(self.vedges[et[-1].oend])
            et.append(self.hedges[et[-1].xend])
            if et[-1] == et[0]:
                closed = True
        del et[-1]
        return et

    def get_PD(self):
        if len(self.crossings) == 0:
            return []
        else:
            dummy_cr = Crossing(self.hedges[0], self.vedges[0])
            et, et_divided = self.edge_trace(), []
         
            et_divided.append([(et[-1], dummy_cr)])
            for e in et:
                et_divided[-1].append((e, dummy_cr))
                e_div = []
                for cr in self.crossings:
                    if e in [cr.hedge, cr.vedge]:
                        e_div.append([(e, cr)])
                et_divided += e_div[::e.direction]
                    
#            for i, l in enumerate(et_divided):
#                print(f"{i}: {l}\n")
        
            modulo = len(et_divided) - 1
            cr_data = [{'cr': x, 'h-dir': None, 'code': {'w': None, 'n': None, 'e': None, 's': None}} for x in self.crossings]
            for i, l in enumerate(et_divided):
                nums = ((i-1)%modulo, i%modulo)
                e, x = l[0]
                for d in cr_data:
                    if d['cr'] == x:
                        if e.hv == 'h':
                            d['code']['e'], d['code']['w'] = nums[::e.direction]
                            d['h-dir'] = e.direction
                        else:
                            d['code']['n'], d['code']['s'] = nums[::e.direction]
            PD = []
            for d in cr_data:
                w, n, e, s = d['code']['w'], d['code']['n'], d['code']['e'], d['code']['s']
                code = [w,n,e,s] if d['h-dir'] == -1 else [e,s,w,n]
                PD.append(code)
            return reduce(PD)
#-----------------------------
def reduce_once(PD):
    for i, l in enumerate(PD):
        if len(set(l)) < len(l):
            pair = [v for v in l if l.count(v) == 1]
            del PD[i]
            PD = [list(map(lambda x: min(pair) if x==max(pair) else x, l)) for l in PD]
            break
    return PD

def reduce(PD):
    prev = len(PD)
    flag = True
    while flag and prev > 0:
        PD = reduce_once(PD)
        if len(PD) == prev:
            flag = False
        else:
            prev = len(PD)
    flatten = list(set(sum(PD, [])))
    return [list(map(lambda x: flatten.index(x), l)) for l in PD]

## Scratch

In [ ]:
#mg = Grid(*[[3, 2, 5, 4, 7, 6, 0, 1], [0, 4, 3, 6, 5, 1, 2, 7]]); 
flag = True
while flag:
    G = generate_random_grid(20)
    sG = set(sum(simplify_grid(G), []))
    if number_of_components(G) == 1 and sG != {0,1}:
        flag = False

#---
mg = Grid(*G); mg.draw(); #print(simplify_grid(G))
#print(mg.edge_trace())
PD = mg.get_PD(); #print(PD)
print(reduce(PD))
#---
# print("\n")
hfk = pd_to_hfk(PD)
#hfk = pd_to_hfk(list(sum(PD, [])))
if hfk['fibered'] == 'True':
    print(f"{hfk['seifert_genus']}\n{hfk['ranks']}")
else:
    print("NO FIBERED")
#    for dgs, r in ast.literal_eval(hfk['ranks']):
#        if hfk['seifert_genus'] == dgs[0]:
#            print(f"lambda: {dgs[1]}")

In [8]:
Count, N, pbar = 0, 5, tqdm(total=N)

Glist = []
while Count < N:
    G = generate_random_grid(20)
    sG = set(sum(simplify_grid(G), []))
    if number_of_components(G) == 1 and sG != {0,1}:
        mg = Grid(*G)
        PD = mg.get_PD(); #print(PD)
        #---
        hfk = pd_to_hfk(PD)
        if hfk['fibered']:
            for mgr, agr in hfk['ranks'].keys():
                if mgr == hfk['seifert_genus']:
                    lmd = agr
            if mgr > lmd:
                Glist.append({'grid': G, 'seifert_genus': mgr, 'lmd': agr})
                Count +=1
                pbar.update(1)
pbar.close()

DF = pd.DataFrame(Glist); display(DF)

  0%|          | 0/5 [00:00<?, ?it/s]

,grid,seifert_genus,lmd
0,"[[9, 17, 11, 1, 4, 2, 12, 3, 0, 10, 19, 16, 13...",2,1
1,"[[1, 0, 18, 16, 9, 11, 17, 6, 13, 12, 15, 3, 2...",1,0
2,"[[5, 7, 12, 18, 15, 13, 4, 16, 2, 9, 3, 14, 17...",4,3
3,"[[0, 3, 17, 19, 16, 9, 4, 10, 12, 18, 1, 5, 8,...",2,1
4,"[[16, 11, 9, 4, 2, 19, 17, 5, 10, 0, 1, 13, 18...",2,1
